In [15]:
import ReportTimerange as rtr
import ReportMailer as rm
import GuestbookDbConnect as gdb

# Install a conda package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install xlsxwriter

# Reporting Period #
* PD - yesterday
* PW - previous week
* PM - previous month
* PQ - previous quarter
* PY - previous calendar year
* YTD year to date
* WTD current week to date
* TODAY today
* P90 - previous 90days

In [16]:
acronyms = [ "PD", "PW", "PM", "PQ", "PY", "YTD", "WTD", "TODAY", "P90"]
friendly = [ "Prior Day", "Prior Week", "Prior Month", "Prior Quarter", "Prior Year", "Year to Date", "Week to Date", "Today", "Previous 90 days"]

def toPeriodFriendly(INTERVAL):
    for n in range(0, len(acronyms)):
      if INTERVAL==acronyms[n]:
         return friendly[n]
    return "Unknown Period"

In [17]:
import os
import pandas as pd

path = "/home/gbadmin/jupy-notebooks/Reports/inqueue"
dir_list = os.listdir(path)

if len(dir_list)>0:
  filename = "inqueue/{}".format(dir_list[0])
  print(filename)

  rep = pd.read_json(filename, lines=True)

  for _ , row in rep.iterrows():    
    _INTERVAL = row['interval']
    _TYPE = row['type']
    if (_TYPE=='IND'):
       _CLIENTID = row['client']
       _APPUSER  = row['appuser']
       print(_INTERVAL)
       print(_TYPE)
       print(_CLIENTID)
       print("AppUser {}".format(_APPUSER))
       idperson = _CLIENTID
       # delete the file
       os.remove(filename)
    else:
      print("Non-ID Type")
      exit()
       
else:
  print("NOTHING TO DO")
  exit()

inqueue/report-20221020_170228.json
YTD
IND
673
AppUser 1


In [4]:
from pandas import ExcelWriter
from pandas import ExcelFile
import xlsxwriter
#import datetime
from datetime import datetime

def createSpreadsheetAndMailIt(collections, reportname, recipients, subject, body):
  now = datetime.now().strftime('%Y-%m-%d_%H%M%S')
  filename = reportname + "-" + now + ".xlsx"
  #writer = ExcelWriter(filename)
  writer = ExcelWriter(filename, engine='xlsxwriter')
  workbook = writer.book  
  print("Writing dataframe to Excel file {0}".format(filename))
  for collection in collections:
    print("Writing {} to spreadsheet".format(collection["name"]))
    collection["dataframe"].to_excel(writer, sheet_name=collection["name"], index=False)
    worksheet = writer.sheets[collection["name"]]
    if 'colwidths' in collection:
      colwidths = collection['colwidths']
      print("colwidths={}".format(colwidths))
      for n in range(0, len(colwidths)):
        col = str(chr(65 + n))
        worksheet.set_column('{}:{}'.format(col,col), colwidths[n])
    else:
      worksheet.set_column('A:A', 30)
  writer.save()
  for recipient in recipients: 
    rm.mailer(recipient, subject, body, filename)
  print("Excel/Email Done!")

In [5]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import GuestbookDbConnect as gdb

conn = gdb.guestbookDbConnect()

Welcome to Jupyter Notebook.  You are connected to the Opportunity House guestbook database!


In [6]:
import ReportTimerange as rtr
trange = rtr.timerange(_INTERVAL)
print(trange)

collections = []
summaries = []

collection = {}
collection["name"] = "Outline"

df = pd.DataFrame({"Report Tab":[ \
              "Client Information", \
              "Visit Counts", \
              "Visit Counts Sub-period", \
              "Services Delivered", \
              "Services Delivered Sub-period", \
              "Housing", \
              "Housing by Sub-Period", \
              "Suspensions", \
              "Incarcerations"
              ], \
                   "Description":[ \
              "client information", \
              "number of visits", \
              "number of visits by sub-period", \
              "services delivered", \
              "services delivered by sub-period", \
              "housing", \
              "housing by sub-period", \
              "suspension history", \
              "incarceration history"
                                 ]})

collection["dataframe"] = df 
collection["colwidths"] = [30, 100]
collections.append(collection)

df.head(1000)

[datetime.datetime(2022, 1, 1, 0, 0), datetime.datetime(2022, 10, 20, 16, 59, 28, 248502), 'YYYY-MM']


Report Tab                       Description
0             Client Information                client information
1                   Visit Counts                  number of visits
2        Visit Counts Sub-period    number of visits by sub-period
3             Services Delivered                services delivered
4  Services Delivered Sub-period  services delivered by sub-period
5                        Housing                           housing
6          Housing by Sub-Period             housing by sub-period
7                    Suspensions                suspension history
8                 Incarcerations             incarceration history

## Client Information ##

In [7]:
from datetime import datetime
now = datetime.now(tz=None)

query = "SELECT firstname, lastname, ethnicity_id, gender_id, \
         language_id, veteran_id, credits \
         FROM guestbook_person WHERE idperson={}".format(idperson)

data = pd.read_sql(query, conn)

#print(data)

firstname = data.iloc[0,0]
lastname  = data.iloc[0,1]
fullname = "{} {}".format(firstname,lastname)
ethnicity_id  = data.iloc[0,2]
gender_id    = data.iloc[0,3]
language_id  = data.iloc[0,4]
veteran_id   = data.iloc[0,5]
credits   = data.iloc[0,6]

print("Report for {} {}".format(firstname, lastname))
print("client ID is {}\n".format(idperson))

if (ethnicity_id==1):
    ethnicity  = "Unknown"
elif (ethnicity_id==2):
    ethnicity  = "Asian"
elif (ethnicity_id==3):
     ethnicity  = "Black"
elif (ethnicity_id==4):
      ethnicity  = "Hispanic"
elif (ethnicity_id==5):
     ethnicity  = "Native American"
elif (ethnicity_id==6):
      ethnicity  = "White"
        
if (gender_id==1):
    gender = "Unknown"
elif (gender_id==2):
    gender = "Male"
elif (gender_id==3):
    gender = "Female"
        
if (language_id==1):
   language = "Preferred language unknown"
elif (language_id==2):
   language = "Preferred language is English"
elif (language_id==3):
   language = "Preferred language is Spanish"
        
if (veteran_id==1):
    veteran = "Veteran status is unknown"
elif (veteran_id==2):
    veteran = "Is a veteran of the US Armed Forces"
elif (veteran_id==3):
    veteran = "Is not a veteran of the US Armed Forces"
        
print("Client has {} credits in the OpportunityHouse Bank\n".format(credits))

query = "SELECT min(timestamp)AS firstdate, max(timestamp) AS lastdate \
         FROM guestbook_personsnapshot \
         WHERE person_id={}".format(idperson)

data = pd.read_sql(query, conn)

firstvisit = "{} ({} days ago)".format(data.iloc[0,0].strftime('%m/%d/%Y'), (now- data.iloc[0,0].replace(tzinfo=None)).days)
lastvisit  = "{} ({} days ago)".format(data.iloc[0,1].strftime('%m/%d/%Y'), (now- data.iloc[0,1].replace(tzinfo=None)).days)

collection = {}
collection["name"] = "Summary Info"

df = pd.DataFrame({"Attribute":[ \
              "Name", \
              "Client ID", \
              "Ethnicity", \
              "Gender", \
              "Language", \
              "Veteran", \
              "First visit", \
              "Last visit", \
              "OppHouse credits"
              ], \
                   "Value":[ \
              fullname, \
              idperson, \
              ethnicity, \
              gender, \
              language, \
              veteran, \
              firstvisit, \
              lastvisit, \
              credits
              ]})

collection["dataframe"] = df 
collection["colwidths"] = [30, 100]
collections.append(collection)

print(df)

df.head(1000)

Report for AARON SPIVEY
client ID is 673

Client has 8 credits in the OpportunityHouse Bank

          Attribute                                    Value
0              Name                             AARON SPIVEY
1         Client ID                                      673
2         Ethnicity                                    White
3            Gender                                     Male
4          Language            Preferred language is English
5           Veteran  Is not a veteran of the US Armed Forces
6       First visit               01/30/2019 (1359 days ago)
7        Last visit                 09/30/2022 (20 days ago)
8  OppHouse credits                                        8


Attribute                                    Value
0              Name                             AARON SPIVEY
1         Client ID                                      673
2         Ethnicity                                    White
3            Gender                                     Male
4          Language            Preferred language is English
5           Veteran  Is not a veteran of the US Armed Forces
6       First visit               01/30/2019 (1359 days ago)
7        Last visit                 09/30/2022 (20 days ago)
8  OppHouse credits                                        8

## Total visits in report period ##

In [8]:
summary = {}
summary["name"] = "Total client visits in report period"

query = "SELECT min(timestamp)AS earliest_visit, max(timestamp) AS latest_visit, \
         COUNT(DISTINCT idsnapshot) AS total_visits FROM guestbook_personsnapshot \
         WHERE person_id={} AND timestamp BETWEEN '{}' AND '{}'".format(idperson, trange[0], trange[1])

#print(query)

data = pd.read_sql(query, conn)

summary["count"] = data.iloc[0,2]
summaries.append(summary)
print(summaries)

data.head(100)

[{'name': 'Total client visits in report period', 'count': 13}]


earliest_visit                     latest_visit  \
0 2022-01-31 14:04:49.373803+00:00 2022-09-30 13:45:08.142984+00:00   

   total_visits  
0            13

# Visits by sub-interval within the report period #
* Statistics focused on the regularity of client visits over the reporting period.

collection = {}
collection["name"] = "Distinct Visits"

query = "SELECT to_char(timestamp,'{}') AS period,  \
         COUNT(DISTINCT snapshot.idsnapshot) AS visits \
         FROM guestbook_personsnapshot snapshot \
         JOIN guestbook_person person ON person.idperson={} \
         WHERE snapshot.person_id = {} AND timestamp BETWEEN '{}' AND '{}' \
         GROUP BY period".format(trange[2], idperson, idperson, trange[0], trange[1])
print(query)

data = pd.read_sql(query, conn)
#data.head(1000)

persons = data.pivot_table('visits', index=['period']).fillna(0).astype(int).reset_index('period')

collection["dataframe"] = persons 
collection["colwidths"] = [30, 20]
collections.append(collection)
persons.head(1000)

## Services delivered ##

In [9]:
query = "SELECT \
         left(service.name, strpos(service.name, '/') - 1) AS servicename, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_personservicerequest servicerequest ON servicerequest.connection_id=snapshot.idsnapshot \
         JOIN guestbook_service service ON service.idservice=servicerequest.service_id \
         WHERE snapshot.person_id = {} AND timestamp BETWEEN '{}' AND '{}'  AND service.points<=0 \
         GROUP BY servicename \
         ORDER BY total desc".format(idperson, trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)


for n in range(0, data.shape[0]):  
  summary = {}
  summary["name"]  = data.iloc[n,0] + " services delivered."
  summary["count"] = data.iloc[n,1]
  summaries.append(summary)
print(summaries)
data.head(1000)

SELECT          left(service.name, strpos(service.name, '/') - 1) AS servicename, COUNT(*) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_personservicerequest servicerequest ON servicerequest.connection_id=snapshot.idsnapshot          JOIN guestbook_service service ON service.idservice=servicerequest.service_id          WHERE snapshot.person_id = 673 AND timestamp BETWEEN '2022-01-01 00:00:00' AND '2022-10-20 16:59:28.248502'  AND service.points<=0          GROUP BY servicename          ORDER BY total desc
[{'name': 'Total client visits in report period', 'count': 13}, {'name': 'Meals services delivered.', 'count': 8}, {'name': 'Shower services delivered.', 'count': 7}, {'name': 'Clothing services delivered.', 'count': 4}, {'name': 'Laundry services delivered.', 'count': 1}, {'name': 'Nurse services delivered.', 'count': 1}]


servicename  total
0       Meals      8
1      Shower      7
2    Clothing      4
3     Laundry      1
4       Nurse      1

# Services delivered by sub-interval #

collection = {}
collection["name"] = "Services"

#query = "SELECT to_char(timestamp,'{}') AS period, \
#         service.name AS servicename, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
#         JOIN guestbook_personservicerequest servicerequest ON servicerequest.connection_id=snapshot.idsnapshot \
#         JOIN guestbook_service service ON service.idservice=servicerequest.service_id \
#         WHERE timestamp BETWEEN '{}' AND '{}' \
#         GROUP BY period, servicename".format(trange[2], trange[0], trange[1])

query = "SELECT to_char(timestamp,'{}') AS period, \
         left(service.name, strpos(service.name, '/') - 1) AS servicename, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_personservicerequest servicerequest ON servicerequest.connection_id=snapshot.idsnapshot \
         JOIN guestbook_service service ON service.idservice=servicerequest.service_id \
         WHERE snapshot.person_id={} AND timestamp BETWEEN '{}' AND '{}' AND service.points<=0 \
         GROUP BY period, servicename".format(trange[2], idperson, trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

services = data.pivot_table('total', index=['period'], columns='servicename').fillna(0).astype(int).reset_index('period')
collection["dataframe"] = services 
collection["colwidths"] = [30, 20, 20, 20, 20, 20, 20, 20]
collections.append(collection)

services.head(1000)

# Housing totals#

In [10]:
query = "SELECT  \
         left(housing.name, strpos(housing.name, '(') - 1)  AS housing_type, COUNT(*) AS nights_spent FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
         WHERE snapshot.person_id={} AND timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 \
         GROUP BY housing_type".format(idperson, trange[0], trange[1])

#print(query)

data = pd.read_sql(query, conn)

for n in range(0, data.shape[0]):  
  summary = {}
  summary["name"]  = " Days spent in " + data.iloc[n,0] 
  summary["count"] = data.iloc[n,1]
  summaries.append(summary)
#print(summaries)

data.head(1000)

housing_type  nights_spent
0  Unsheltered             10

# Housing by sub-interval within the report period#
* Answers to the prompt 'Where did you spend last night?'

collection = {}
collection["name"] = "Housing"

#query = "SELECT to_char(timestamp,'{}') AS period, \
#         housing.name AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
#         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
#         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
#         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 \
#         GROUP BY period, hresponse".format(trange[2], trange[0], trange[1])

query = "SELECT to_char(timestamp,'{}') AS period, \
         left(housing.name, strpos(housing.name, '(') - 1) AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
         WHERE snapshot.person_id={} AND timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 \
         GROUP BY period, hresponse".format(trange[2], idperson, trange[0], trange[1])

#print(query)

data = pd.read_sql(query, conn)

housing = data.pivot_table('total', index=['period'], columns='hresponse').fillna(0).astype(int).reset_index('period')
#housing = housing.fillna(0)
collection["dataframe"] = services 
collection["colwidths"] = [30, 20, 20, 20, 20, 20, 20, 20]
collections.append(collection)

services.head(1000)

# Suspensions #

In [11]:
query = "SELECT starttime AS suspension_starttime, (endtime-starttime) AS days, suspensionreason.name AS reason FROM guestbook_suspension \
         JOIN guestbook_suspensionreason AS suspensionreason ON idsuspensionreason = reason_id \
         WHERE person_id={} AND starttime BETWEEN '{}' AND '{}'".format(idperson, trange[0], trange[1])


data = pd.read_sql(query, conn)

collection["dataframe"] = data
collections.append(collection)

data.head(1000)



Empty DataFrame
Columns: [suspension_starttime, days, reason]
Index: []

# Incarcerations #

In [12]:
collection = {}
collection["name"] = "Incarcerations"


#query = "SELECT to_char(timestamp,'{}') AS period, \
#         housing.name AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
#         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
#         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
#         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 \
#         GROUP BY period, hresponse".format(trange[2], trange[0], trange[1])

query = "SELECT COUNT(DISTINCT(idsnapshot)) \
         FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
         WHERE snapshot.person_id={} AND survey.object_id=2 AND timestamp BETWEEN '{}' AND '{}' \
         AND prompt_id=4".format(idperson, trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

#housing = data.pivot_table('total', index=['period'], columns='hresponse').fillna(0).astype(int).reset_index('period')
collection["dataframe"] = data 
collection["colwidths"] = [30, 30, 30]
collections.append(collection)
data.head(1000)

SELECT COUNT(DISTINCT(idsnapshot))          FROM guestbook_personsnapshot snapshot         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot          WHERE snapshot.person_id=673 AND survey.object_id=2 AND timestamp BETWEEN '2022-01-01 00:00:00' AND '2022-10-20 16:59:28.248502'          AND prompt_id=4


count
0      0

# Summary Counts #

In [13]:
collection = {}
summ = pd.DataFrame.from_dict(summaries)
collection["name"] = "Summary"
collection["dataframe"] = summ
collection["colwidths"] = [30, 60]
collections.insert(1, collection)
#collections.append(collection)
summ.head(1000)

count                                  name
0     13  Total client visits in report period
1      8             Meals services delivered.
2      7            Shower services delivered.
3      4          Clothing services delivered.
4      1           Laundry services delivered.
5      1             Nurse services delivered.
6     10            Days spent in Unsheltered

In [14]:
#import os
#import pandas as pd
#import GuestbookDbConnect as gdb
#conn = gdb.guestbookDbConnect()

_REPORTNAME = _INTERVAL
_SUBJECT = "Opportunity House - Client Report  for {} {}, {}".format(firstname, lastname, toPeriodFriendly(_INTERVAL))
_BODY    = "Spreadsheet (attached) with reports for period {} through {}.".format(trange[0], trange[1])

#_EMAIL_RECIPIENT = ['cprice9739@carolina.rr.com', 'jahood1@yahoo.com']
#_EMAIL_RECIPIENT = ['cprice9739@carolina.rr.com', 'pastor@opphouse.net']
#_EMAIL_RECIPIENT = ['cprice9739@carolina.rr.com']

query = "SELECT id, email FROM guestbook_appuser WHERE id={}".format(_APPUSER)

#print(query)
data = pd.read_sql(query, conn)
_EMAIL_RECIPIENT = {data.iloc[0,1]}
print(">>{}<<".format(_EMAIL_RECIPIENT))

createSpreadsheetAndMailIt(collections, _REPORTNAME, _EMAIL_RECIPIENT, _SUBJECT, _BODY)

>>{'jahood1@yahoo.com'}<<
Writing dataframe to Excel file YTD-2022-10-20_165952.xlsx
Writing Outline to spreadsheet
colwidths=[30, 100]
Writing Summary to spreadsheet
colwidths=[30, 60]
Writing Summary Info to spreadsheet
colwidths=[30, 100]
Writing Summary Info to spreadsheet
colwidths=[30, 100]
Writing Incarcerations to spreadsheet
colwidths=[30, 30, 30]
Emailer Done!
Excel/Email Done!
